# Gold Price Prediction with News Integration

This notebook predicts gold price direction (up/down) and price range using:
- Historical gold price data from Kaggle
- Google RSS news feeds for latest news sentiment
- Real-time gold price API for current prices

## Prediction Goals
1. Predict whether gold price will go UP or DOWN
2. Predict the price range/amount of change


## 0. Configuration

Set your Kaggle API token here (or use environment variables in Colab)


In [ ]:
# Configuration
KAGGLE_API_TOKEN = "KGAT_b352cb91c46b038224e3d90adb8d8c32"
ALPHA_VANTAGE_API_KEY = "EF6488BOEZN0B69R"

# Set environment variable for Kaggle
import os
os.environ['KAGGLE_API_TOKEN'] = KAGGLE_API_TOKEN


In [ ]:
## 1. Setup and Installation


# Install required packages
!pip install kagglehub pandas numpy scikit-learn matplotlib seaborn feedparser requests beautifulsoup4 textblob


In [ ]:
# Download Kaggle dataset using kagglehub
import kagglehub

path = kagglehub.dataset_download("sid321axn/gold-price-prediction-dataset")
print("Path to dataset files:", path)


## 2. Import Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# For RSS feeds and news
import feedparser
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob

# For API calls
import json
import time

# For machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error, mean_squared_error

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


## 3. Load and Explore Kaggle Dataset


In [ ]:
# Load the dataset
# Adjust the path based on your downloaded dataset structure
dataset_path = path
print(f"Dataset located at: {dataset_path}")

# List files in the dataset
if os.path.exists(dataset_path):
    files = os.listdir(dataset_path)
    print("Files in dataset:", files)
    
    # Load the main CSV file (adjust filename as needed)
    csv_files = [f for f in files if f.endswith('.csv')]
    if csv_files:
        df_gold = pd.read_csv(os.path.join(dataset_path, csv_files[0]))
        print(f"\nLoaded: {csv_files[0]}")
        print(f"Shape: {df_gold.shape}")
        print(f"\nFirst few rows:")
        print(df_gold.head())
        print(f"\nColumn names: {df_gold.columns.tolist()}")
        print(f"\nData types:\n{df_gold.dtypes}")
        print(f"\nBasic statistics:\n{df_gold.describe()}")
    else:
        print("No CSV files found in dataset")
else:
    print("Dataset path not found. Please check the path.")


## 4. Google RSS News Integration


In [ ]:
def get_news_sentiment(query="gold price", max_results=20):
    """
    Fetch news from Google RSS feeds and analyze sentiment
    """
    # Google News RSS feed URL
    rss_url = f"https://news.google.com/rss/search?q={query}&hl=en-US&gl=US&ceid=US:en"
    
    try:
        feed = feedparser.parse(rss_url)
        news_items = []
        
        for entry in feed.entries[:max_results]:
            title = entry.get('title', '')
            summary = entry.get('summary', '')
            published = entry.get('published', '')
            link = entry.get('link', '')
            
            # Combine title and summary for sentiment analysis
            text = f"{title} {summary}"
            blob = TextBlob(text)
            sentiment_score = blob.sentiment.polarity  # Range: -1 to 1
            
            news_items.append({
                'title': title,
                'summary': summary,
                'published': published,
                'link': link,
                'sentiment': sentiment_score
            })
        
        return pd.DataFrame(news_items)
    
    except Exception as e:
        print(f"Error fetching news: {e}")
        return pd.DataFrame()

def aggregate_news_sentiment(news_df):
    """
    Aggregate news sentiment for a given day/period
    """
    if news_df.empty:
        return {
            'avg_sentiment': 0.0,
            'positive_count': 0,
            'negative_count': 0,
            'news_count': 0
        }
    
    avg_sentiment = news_df['sentiment'].mean()
    positive_count = (news_df['sentiment'] > 0.1).sum()
    negative_count = (news_df['sentiment'] < -0.1).sum()
    
    return {
        'avg_sentiment': avg_sentiment,
        'positive_count': positive_count,
        'negative_count': negative_count,
        'news_count': len(news_df)
    }

# Fetch current gold-related news
print("Fetching latest gold price news...")
news_df = get_news_sentiment("gold price OR gold market OR gold trading", max_results=30)
print(f"\nFetched {len(news_df)} news items")

if not news_df.empty:
    print("\nSample news items:")
    print(news_df[['title', 'sentiment']].head(10))
    
    # Aggregate sentiment
    sentiment_stats = aggregate_news_sentiment(news_df)
    print(f"\nNews Sentiment Statistics:")
    print(f"Average Sentiment: {sentiment_stats['avg_sentiment']:.3f}")
    print(f"Positive News: {sentiment_stats['positive_count']}")
    print(f"Negative News: {sentiment_stats['negative_count']}")
    print(f"Total News: {sentiment_stats['news_count']}")


## 5. Free Gold Price API Integration


In [ ]:
def get_gold_price_api(api_key=None):
    """
    Fetch current gold price using free APIs
    Options: Metal-API, Alpha Vantage, or GoldAPI.io (free tier available)
    """
    current_price = None
    price_data = {}
    
    # Method 1: Metal-API (Free tier: 200 requests/month, no API key needed for basic)
    try:
        url = "https://api.metalpriceapi.com/v1/latest?api_key=YOUR_API_KEY&base=USD&currencies=XAU"
        # Alternative: Use exchangerate-api.com for free gold prices
        url_alt = "https://api.exchangerate-api.com/v4/latest/XAU"
        
        response = requests.get(url_alt, timeout=10)
        if response.status_code == 200:
            data = response.json()
            # Note: This API structure may vary, adjust accordingly
            current_price = data.get('rates', {}).get('USD', None)
            if current_price:
                current_price = 1 / current_price  # Convert from XAU to USD per ounce
            price_data['source'] = 'exchangerate-api'
    except Exception as e:
        print(f"Method 1 failed: {e}")
    
    # Method 2: GoldPrice.org API (free, no key required)
    if current_price is None:
        try:
            url = "https://api.goldprice.org/gpapi/v2/quotes/USD"
            headers = {
                'User-Agent': 'Mozilla/5.0',
                'Accept': 'application/json'
            }
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                data = response.json()
                # Parse response based on actual API structure
                current_price = data.get('price', None)
                price_data['source'] = 'goldprice.org'
        except Exception as e:
            print(f"Method 2 failed: {e}")
    
    # Method 3: Alpha Vantage (free tier available)
    if current_price is None and api_key:
        try:
            url = f"https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=XAU&to_currency=USD&apikey={api_key}"
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                rate = data.get('Realtime Currency Exchange Rate', {})
                current_price = float(rate.get('5. Exchange Rate', 0))
                price_data['source'] = 'alpha-vantage'
        except Exception as e:
            print(f"Method 3 failed: {e}")
    
    # Use the API key from configuration if available
    if current_price is None and 'ALPHA_VANTAGE_API_KEY' in globals() and ALPHA_VANTAGE_API_KEY:
        try:
            url = f"https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=XAU&to_currency=USD&apikey={ALPHA_VANTAGE_API_KEY}"
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                rate = data.get('Realtime Currency Exchange Rate', {})
                current_price = float(rate.get('5. Exchange Rate', 0))
                price_data['source'] = 'alpha-vantage'
        except Exception as e:
            print(f"Method 3 (from config) failed: {e}")
    
    if current_price:
        price_data['current_price'] = current_price
        price_data['timestamp'] = datetime.now().isoformat()
        print(f"Current Gold Price (USD/oz): ${current_price:.2f} from {price_data.get('source', 'unknown')}")
    else:
        print("Warning: Could not fetch current gold price from APIs")
        # Use a placeholder or last known price
        current_price = 2000.0  # Placeholder
        price_data['current_price'] = current_price
        price_data['source'] = 'placeholder'
    
    return price_data

# Fetch current gold price
print("Fetching current gold price...")
current_gold_data = get_gold_price_api(api_key=ALPHA_VANTAGE_API_KEY if 'ALPHA_VANTAGE_API_KEY' in globals() else None)
print(f"\nGold Price Data: {current_gold_data}")


## 6. Data Preprocessing


In [ ]:
def preprocess_gold_data(df):
    """
    Preprocess the gold price dataset
    """
    df_processed = df.copy()
    
    # Convert date column to datetime (adjust column name as needed)
    date_col = None
    for col in df_processed.columns:
        if 'date' in col.lower() or 'time' in col.lower():
            date_col = col
            break
    
    if date_col:
        df_processed[date_col] = pd.to_datetime(df_processed[date_col], errors='coerce')
        df_processed = df_processed.sort_values(date_col).reset_index(drop=True)
    
    # Identify price column (usually contains 'price', 'close', or 'value')
    price_col = None
    for col in df_processed.columns:
        if 'price' in col.lower() or 'close' in col.lower() or 'value' in col.lower():
            price_col = col
            break
    
    # Create features
    if price_col and df_processed[price_col].dtype in ['float64', 'int64']:
        # Price change
        df_processed['price_change'] = df_processed[price_col].diff()
        df_processed['price_change_pct'] = df_processed[price_col].pct_change() * 100
        
        # Moving averages
        df_processed['ma_7'] = df_processed[price_col].rolling(window=7).mean()
        df_processed['ma_30'] = df_processed[price_col].rolling(window=30).mean()
        
        # Price volatility
        df_processed['volatility'] = df_processed[price_col].rolling(window=7).std()
        
        # Price direction (target for classification)
        df_processed['direction'] = (df_processed[price_col].shift(-1) > df_processed[price_col]).astype(int)
        
        # Price range for next period (target for regression)
        df_processed['next_price'] = df_processed[price_col].shift(-1)
        # Create a temporary dataframe for min/max calculation
        temp_df = pd.DataFrame({
            'current': df_processed[price_col],
            'next': df_processed['next_price']
        })
        df_processed['price_range_low'] = temp_df[['current', 'next']].min(axis=1)
        df_processed['price_range_high'] = temp_df[['current', 'next']].max(axis=1)
        df_processed['price_range'] = df_processed['price_range_high'] - df_processed['price_range_low']
    
    # Handle missing values - use forward and backward fill
    df_processed = df_processed.fillna(method='bfill').fillna(method='ffill')
    # Drop any remaining NaN rows
    df_processed = df_processed.dropna()
    
    return df_processed, price_col, date_col

# Preprocess the dataset
if 'df_gold' in locals() and not df_gold.empty:
    df_processed, price_col, date_col = preprocess_gold_data(df_gold)
    print(f"Processed dataset shape: {df_processed.shape}")
    print(f"Price column: {price_col}")
    print(f"Date column: {date_col}")
    print(f"\nProcessed data head:")
    print(df_processed.head())
else:
    print("Please load the dataset first in section 3")


## 7. Feature Engineering with News and API Data


In [ ]:
def create_feature_matrix(df_processed, news_sentiment=None, current_api_price=None):
    """
    Create feature matrix combining historical data, news sentiment, and API price
    """
    features = []
    
    # Historical price features
    if price_col and price_col in df_processed.columns:
        feature_cols = [
            price_col,
            'price_change', 'price_change_pct',
            'ma_7', 'ma_30', 'volatility'
        ]
        
        # Add available features
        available_features = [col for col in feature_cols if col in df_processed.columns]
        X = df_processed[available_features].values
        
        # Add news sentiment features (if available)
        if news_sentiment:
            sentiment_features = [
                news_sentiment['avg_sentiment'],
                news_sentiment['positive_count'],
                news_sentiment['negative_count'],
                news_sentiment['news_count']
            ]
            # Repeat sentiment for each row (or match by date if dates available)
            sentiment_array = np.tile(sentiment_features, (len(X), 1))
            X = np.hstack([X, sentiment_array])
        
        # Add current API price features (if available)
        if current_api_price and 'current_price' in current_api_price:
            api_price = current_api_price['current_price']
            # Compare with last historical price
            last_price = df_processed[price_col].iloc[-1] if len(df_processed) > 0 else api_price
            price_diff = api_price - last_price
            price_diff_pct = (price_diff / last_price) * 100
            
            api_features = np.array([[api_price, price_diff, price_diff_pct]])
            api_features_tiled = np.tile(api_features, (len(X), 1))
            X = np.hstack([X, api_features_tiled])
        
        return X, available_features
    
    return None, []

# Create feature matrix
if 'df_processed' in locals() and not df_processed.empty:
    X, feature_names = create_feature_matrix(
        df_processed,
        news_sentiment=sentiment_stats if 'sentiment_stats' in locals() else None,
        current_api_price=current_gold_data if 'current_gold_data' in locals() else None
    )
    
    if X is not None:
        print(f"Feature matrix shape: {X.shape}")
        print(f"Feature names: {feature_names}")
        print(f"\nFeature matrix sample:")
        print(X[:5])


## 8. Model Training - Direction Prediction (Up/Down)


In [ ]:
def train_direction_model(X, y_direction):
    """
    Train a model to predict price direction (up/down)
    """
    # Remove rows where target is NaN
    valid_mask = ~np.isnan(y_direction)
    X_clean = X[valid_mask]
    y_clean = y_direction[valid_mask].astype(int)
    
    if len(X_clean) == 0:
        print("No valid data for training")
        return None, None, None
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.2, random_state=42, stratify=y_clean
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train Random Forest Classifier
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Direction Prediction Accuracy: {accuracy:.4f}")
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Down', 'Up']))
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_names[:len(model.feature_importances_)] if 'feature_names' in globals() else [f'feature_{i}' for i in range(len(model.feature_importances_))],
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop Features:")
    print(feature_importance.head(10))
    
    return model, scaler, feature_importance

# Train direction model
if 'X' in locals() and X is not None and 'df_processed' in locals():
    if 'direction' in df_processed.columns:
        y_direction = df_processed['direction'].values
        
        direction_model, direction_scaler, feature_importance = train_direction_model(X, y_direction)
        
        if direction_model is not None:
            print("\nDirection model trained successfully!")
    else:
        print("Direction column not found. Creating it...")
        # Create direction column if it doesn't exist
        if price_col in df_processed.columns:
            df_processed['direction'] = (df_processed[price_col].shift(-1) > df_processed[price_col]).astype(int)
            y_direction = df_processed['direction'].values
            direction_model, direction_scaler, feature_importance = train_direction_model(X, y_direction)
else:
    print("Please complete data preprocessing first")


## 9. Model Training - Price Range Prediction


In [ ]:
def train_range_model(X, y_price):
    """
    Train a model to predict price range (how much the price will change)
    """
    # Remove rows where target is NaN
    valid_mask = ~np.isnan(y_price)
    X_clean = X[valid_mask]
    y_clean = y_price[valid_mask]
    
    if len(X_clean) == 0:
        print("No valid data for training")
        return None, None, None
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.2, random_state=42
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train Random Forest Regressor for price prediction
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_pred = model.predict(X_test_scaled)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print(f"Price Range Prediction:")
    print(f"Mean Absolute Error: ${mae:.2f}")
    print(f"Root Mean Squared Error: ${rmse:.2f}")
    print(f"Mean Actual Price: ${y_test.mean():.2f}")
    print(f"Mean Predicted Price: ${y_pred.mean():.2f}")
    
    return model, scaler, {'mae': mae, 'rmse': rmse}

# Train range model
if 'X' in locals() and X is not None and 'df_processed' in locals():
    if 'next_price' in df_processed.columns:
        y_price = df_processed['next_price'].values
        
        range_model, range_scaler, range_metrics = train_range_model(X, y_price)
        
        if range_model is not None:
            print("\nPrice range model trained successfully!")
    else:
        print("Next price column not found. Please check data preprocessing.")
else:
    print("Please complete data preprocessing first")


## 10. Make Predictions


In [ ]:
def predict_gold_price(direction_model, direction_scaler, range_model, range_scaler, 
                       latest_features, current_price):
    """
    Make predictions for gold price direction and range
    """
    predictions = {}
    
    # Predict direction
    if direction_model is not None and direction_scaler is not None:
        # Use the latest features for prediction
        latest_features_scaled = direction_scaler.transform([latest_features])
        direction_proba = direction_model.predict_proba(latest_features_scaled)[0]
        direction_pred = direction_model.predict(latest_features_scaled)[0]
        
        predictions['direction'] = 'UP' if direction_pred == 1 else 'DOWN'
        predictions['direction_confidence'] = max(direction_proba) * 100
        predictions['up_probability'] = direction_proba[1] * 100
        predictions['down_probability'] = direction_proba[0] * 100
    
    # Predict price range
    if range_model is not None and range_scaler is not None:
        latest_features_scaled = range_scaler.transform([latest_features])
        predicted_price = range_model.predict(latest_features_scaled)[0]
        
        # Calculate range
        price_change = predicted_price - current_price
        price_change_pct = (price_change / current_price) * 100
        
        # Estimate uncertainty (using model's feature importance as proxy)
        # In production, use prediction intervals or quantile regression
        uncertainty = abs(price_change) * 0.1  # 10% uncertainty estimate
        
        predictions['predicted_price'] = predicted_price
        predictions['current_price'] = current_price
        predictions['expected_change'] = price_change
        predictions['expected_change_pct'] = price_change_pct
        predictions['price_range_low'] = predicted_price - uncertainty
        predictions['price_range_high'] = predicted_price + uncertainty
        predictions['range_span'] = uncertainty * 2
    
    return predictions

# Make predictions
if 'direction_model' in locals() and direction_model is not None:
    if 'X' in locals() and X is not None and len(X) > 0:
        latest_features = X[-1]  # Use most recent data point
        current_price_val = current_gold_data.get('current_price', df_processed[price_col].iloc[-1] if 'df_processed' in locals() and price_col in df_processed.columns else 2000.0)
        
        predictions = predict_gold_price(
            direction_model, direction_scaler,
            range_model if 'range_model' in locals() else None,
            range_scaler if 'range_scaler' in locals() else None,
            latest_features,
            current_price_val
        )
        
        print("\n" + "="*50)
        print("GOLD PRICE PREDICTION RESULTS")
        print("="*50)
        
        if 'direction' in predictions:
            print(f"\nDirection Prediction: {predictions['direction']}")
            print(f"Confidence: {predictions['direction_confidence']:.2f}%")
            print(f"  - Up Probability: {predictions['up_probability']:.2f}%")
            print(f"  - Down Probability: {predictions['down_probability']:.2f}%")
        
        if 'predicted_price' in predictions:
            print(f"\nPrice Prediction:")
            print(f"  Current Price: ${predictions['current_price']:.2f}/oz")
            print(f"  Predicted Price: ${predictions['predicted_price']:.2f}/oz")
            print(f"  Expected Change: ${predictions['expected_change']:.2f} ({predictions['expected_change_pct']:+.2f}%)")
            print(f"\nPredicted Price Range:")
            print(f"  Low: ${predictions['price_range_low']:.2f}/oz")
            print(f"  High: ${predictions['price_range_high']:.2f}/oz")
            print(f"  Range Span: ${predictions['range_span']:.2f}")
        
        print("\n" + "="*50)
    else:
        print("No features available for prediction")
else:
    print("Please train the models first")


## 11. Visualization


In [ ]:
# Visualize gold price trends
if 'df_processed' in locals() and price_col and date_col:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot 1: Price over time
    axes[0, 0].plot(df_processed[date_col], df_processed[price_col], linewidth=2)
    axes[0, 0].set_title('Gold Price Over Time', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Date')
    axes[0, 0].set_ylabel('Price (USD/oz)')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Add prediction point if available
    if 'predictions' in locals() and 'predicted_price' in predictions:
        last_date = df_processed[date_col].iloc[-1]
        next_date = last_date + pd.Timedelta(days=1)
        axes[0, 0].plot(next_date, predictions['predicted_price'], 'ro', markersize=10, label='Prediction')
        axes[0, 0].errorbar(next_date, predictions['predicted_price'], 
                           yerr=[[predictions['predicted_price'] - predictions['price_range_low']],
                                 [predictions['price_range_high'] - predictions['predicted_price']]],
                           fmt='ro', capsize=5, label='Prediction Range')
        axes[0, 0].legend()
    
    # Plot 2: Price changes
    if 'price_change_pct' in df_processed.columns:
        axes[0, 1].plot(df_processed[date_col], df_processed['price_change_pct'], alpha=0.7)
        axes[0, 1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
        axes[0, 1].set_title('Price Change Percentage', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Date')
        axes[0, 1].set_ylabel('Change (%)')
        axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Moving averages
    if 'ma_7' in df_processed.columns and 'ma_30' in df_processed.columns:
        axes[1, 0].plot(df_processed[date_col], df_processed[price_col], label='Price', alpha=0.7)
        axes[1, 0].plot(df_processed[date_col], df_processed['ma_7'], label='MA 7', linewidth=2)
        axes[1, 0].plot(df_processed[date_col], df_processed['ma_30'], label='MA 30', linewidth=2)
        axes[1, 0].set_title('Price with Moving Averages', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Date')
        axes[1, 0].set_ylabel('Price (USD/oz)')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: News sentiment over time (if multiple data points available)
    if 'news_df' in locals() and not news_df.empty:
        sentiment_counts = [sentiment_stats['positive_count'], sentiment_stats['negative_count']]
        labels = ['Positive', 'Negative']
        colors = ['green', 'red']
        axes[1, 1].bar(labels, sentiment_counts, color=colors, alpha=0.7)
        axes[1, 1].set_title('Current News Sentiment Distribution', fontsize=14, fontweight='bold')
        axes[1, 1].set_ylabel('Number of News Items')
        axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
else:
    print("Data not available for visualization")


## 12. Notes and Next Steps

### API Keys Setup (if needed):
1. **Metal-API**: Get free API key from https://metalpriceapi.com
2. **Alpha Vantage**: Get free API key from https://www.alphavantage.co/support/#api-key
3. **GoldAPI.io**: Free tier available at https://www.goldapi.io

### To use API keys in Google Colab:
1. Go to: Runtime > Change runtime type > Environment variables
2. Or use Colab's secret manager
3. Or set in notebook: `os.environ['API_KEY'] = 'your_key'`

### Model Improvements:
- Use more sophisticated models (LSTM, XGBoost, etc.)
- Implement prediction intervals for better uncertainty estimation
- Add more features (economic indicators, other commodity prices)
- Implement time-series cross-validation
- Add ensemble methods

### Data Updates:
- Set up scheduled runs to fetch latest news and API prices
- Implement real-time monitoring
- Create alerts for significant price movements
